In [1]:
import os
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
import matplotlib as plt
import json
import bert

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

# https://www.youtube.com/watch?v=gE-95nFF4Cc 

In [2]:
bert_model = tf.saved_model.load("./bert_en_uncased_L-12_H-768_A-12_2")

In [ ]:
# !gdown --id 1OlcvGWReJMuyYQuOZm149vHWwPtlboR6 --output intent/train.csv
# !gdown --id 1Oi5cRlTybuIF2Fl5Bfsr-KkqrXrdt77w --output intent/valid.csv
# !gdown --id 1ep9H6-HvhB4utJRLVcLzieWNUSG3P_uF --output intent/test.csv